In [1]:
import numpy as np
from astropy.constants import m_p, G, k_B 
import astropy.units as u
from lmfit import Model
from scipy.integrate import simps
import tqdm
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('../my_funcs/')
from pipeline_main import pipeline
from pipeline_1D import to_1D, get_1D_param

In [2]:
datai = pipeline(snap = 446, run = 'sink225core03', sink_id=225)
data = pipeline(snap = 524, run = 'sink225core03', sink_id=225)
data.recalc_L()

Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 4903/4903 [00:06<00:00, 797.26it/s]


Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 4903/4903 [00:05<00:00, 824.19it/s]


Converged mean angular momentum vector after 0 iteration(s)


In [3]:
r_in = 5; r_out = 100; Nr = 100; plot = True; dpi = 100; verbose = 1
if not data.cyl_calculated: data.recalc_L(verbose = 0)

data.Nr = Nr
data.r_bins = np.logspace(np.log10(r_in), np.log10(r_out), Nr) / data.au_length #[au]
data.r_1D = data.r_bins[:-1] + 0.5 * np.diff(data.r_bins) 

def Hp_func(x, Σ, H): return (Σ) / (np.sqrt(2 * np.pi) * H) * np.exp( - x**2 / (2 * H**2)) 

def fit_scaleheight(ρ, h, x0):
    model = Model(Hp_func)
    params = model.make_params(Σ = x0[0], H = x0[1])
    #params['Σ'].min = 0; params['H'].min = 0   # Ensure H is always positive
    result = model.fit(ρ, x = h, params = params)
    fit_params = np.array(list(result.best_values.values()))
    fit_err = np.array([par.stderr for _, par in result.params.items()])
    fit_params[0] *= data.sn.cgs.au ; fit_err[0] *= data.sn.cgs.au

    return np.array([fit_params[0], fit_err[0]]), np.array([fit_params[1], fit_err[1]]) 


densities = {key: [] for key in range(Nr - 1)}
heights = {key: [] for key in range(Nr - 1)}
if verbose > 0: print('Looping through patches to extract densities and heights')


selection_radius = (r_out * 2) / data.au_length
pp = [p for p in data.sn.patches if np.linalg.norm(p.rel_ppos, axis = 0) < selection_radius]
w= np.array([p.level for p in pp]).argsort()[::-1]
sorted_patches = [pp[w[i]] for i in range(len(pp))]
data.sorted_patches1D = sorted_patches.copy()

for p in data.sorted_patches1D:
    nbors = [data.sn.patchid[i] for i in p.nbor_ids if i in data.sn.patchid]
    children = [ n for n in nbors if n.level == p.level + 1]
    leafs = [n for n in children if ((n.position - p.position)**2).sum() < ((p.size)**2).sum()/12]   
    if len(leafs) == 8: continue

    to_extract = np.ones((16,16,16), dtype='bool')
    for lp in leafs: 
        leaf_extent = np.vstack((lp.position - 0.5 * lp.size, lp.position + 0.5 * lp.size)).T
        covered_bool = ~np.all((p.xyz > leaf_extent[:, 0, None, None, None]) & (p.xyz < leaf_extent[:, 1, None, None, None]), axis=0)
        to_extract *= covered_bool 
    p.to_extract1D = to_extract.copy()
    # Radial cut in the bins are saved for later since the cut will be used repeatedly
    p.bin_idx1D = np.digitize(p.cyl_R[to_extract].flatten(), bins = data.r_bins)                                   #Assigning each point to their repective bin 
    if (p.bin_idx1D == Nr).all() or (p.bin_idx1D == 0).all(): continue                                    #If the points are outside of the given bin the index is 0 or 25 (dependent on the length of radial bins)
    for bin in np.unique(p.bin_idx1D):
        if bin == 0 or bin == Nr: continue                                            
        h_idx = np.nonzero(abs(p.cyl_z[to_extract].flatten()[p.bin_idx1D == bin]) < 2 * data.r_1D[bin - 1])        # Now I make a cut only taking cells within 2 * the radial bins. I want the densities directory to be 0-indexet hence bin - 1
        if len(h_idx) == 0: continue                                             
        
        densities[bin - 1].extend(p.var('d')[to_extract].flatten()[p.bin_idx1D == bin][h_idx])
        heights[bin - 1].extend(p.cyl_z[to_extract].flatten()[p.bin_idx1D == bin][h_idx])

for key in densities:
    densities[key] = np.array(densities[key]) * data.cgs_density
    heights[key] = np.array(heights[key]) * data.au_length

data.Σ_1D = np.zeros((Nr - 1, 2))
data.H_1D = np.zeros((Nr - 1, 2))
x0 = np.array([1e3, 7])

if verbose > 0: print('Fitting surface density and scaleheight in each radial bin')
for i in tqdm.tqdm(range(Nr - 1), disable = not data.loading_bar):    
    if len(densities[i]) == 0: 
        print('Empty densities')
        break
    data.Σ_1D[i], data.H_1D[i] = fit_scaleheight(ρ = densities[i], h = heights[i], x0 = x0)
    x0 = np.array([data.Σ_1D[i, 0], data.H_1D[i, 0]])

Looping through patches to extract densities and heights
Fitting surface density and scaleheight in each radial bin


100%|██████████| 99/99 [00:02<00:00, 47.47it/s] 


In [4]:
snapshots = np.arange(datai.sn.iout, data.sn.iout)
time = np.zeros_like(snapshots)
L_evo = np.zeros_like(snapshots)

In [5]:
for i in tqdm.tqdm(range(len(snapshots))):
    print(datai.sn.iout + i)
    data_loop = pipeline(snap = datai.sn.iout + i , run = 'sink225core03', sink_id = 225, loading_bar = False, verbose = 0)
    data_loop.recalc_L(verbose = 0)
    data_loop.to_1D(plot = False, r_in= 5, r_out = 100, Nr = 100, verbose = 0)
    data_loop.get_1D_param(Ω = True, verbose = 0, get_units=False)
    H = data_loop.H_1D[:,0]
    r = data_loop.r_1D * data.sn.scaling.l
    Σ = data_loop.Σ_1D[:,0]
    v_φ = data_loop.vφ_1D[:,0]
    v_kep = data_loop.kepVφ_1D

    L = simps(2 * np.pi * Σ * v_φ * r**2, r); L_kep = simps(2 * np.pi * Σ * v_kep * r**2, r)  
    L_evo[i] = L/L_kep 

    time[i] = data_loop.time - datai.time + 500

  0%|          | 0/78 [00:00<?, ?it/s]

446


  1%|▏         | 1/78 [00:33<42:46, 33.33s/it]

447


  3%|▎         | 2/78 [01:33<1:02:05, 49.02s/it]

...

 97%|█████████▋| 76/78 [1:09:43<01:37, 48.54s/it]

522


 99%|█████████▊| 77/78 [1:10:20<00:45, 45.28s/it]

523


100%|██████████| 78/78 [1:11:04<00:00, 54.67s/it]


In [16]:
time[i]

38989

In [13]:
L_evo[-1] = np.copy(L/L_kep)

In [15]:
L_evo * 100

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])